#Preprocessing-2
-----------------
**Özgün Yargı**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
%matplotlib inline
import os
import pickle
import random
import json

import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tqdm import tqdm
import pandas as pd
from sklearn import model_selection
import shutil

In [ ]:
def get_box_data(data, index):
    meta_data = dict()
    meta_data['height'] = []
    meta_data['label'] = []
    meta_data['left'] = []
    meta_data['top'] = []
    meta_data['width'] = []

    def print_attrs(name, obj):
        vals = []
        if obj.shape[0] == 1:
            vals.append(obj[0][0])
        else:
            for k in range(obj.shape[0]):
                vals.append(int(data[obj[k][0]][0][0]))
        meta_data[name] = vals

    box = data['/digitStruct/bbox'][index]
    data[box[0]].visititems(print_attrs)
    return meta_data

In [ ]:
d = h5py.File('/content/gdrive/MyDrive/digitStruct.mat', 'r')
a = 1
print (get_box_data(d, a))

{'height': [32, 32], 'label': [2, 3], 'left': [77, 98], 'top': [29, 25], 'width': [23, 26]}


In [ ]:
with open('/content/gdrive/MyDrive/image_names.json') as json_file:
    meta_data = json.load(json_file)

In [ ]:
def create_dateset(dataset, data_):

    counter = 0
    data = h5py.File('/content/gdrive/MyDrive/digitStruct.mat', 'r')
    df = []
    for i in tqdm(range(len(data['/digitStruct/name']))):

        if counter == 10000:
          break
        else:      
          meta_data = get_box_data(data, i)
          num_length = len(meta_data['label'])
          if num_length < 6:
              dd = {'filename': data_["image_names"][i], 'len': num_length}
              for i in range(5):
                  dd['num{}'.format(i+1)] = -1
                  dd['bbox{}'.format(i+1)] = (0, 0, 0, 0)
              for i in range(num_length):
                  dd['num{}'.format(i+1)] = int(meta_data['label'][i])
                  dd['bbox{}'.format(i+1)] = (meta_data['left'][i], meta_data['top'][i], meta_data['width'][i], meta_data['height'][i])
          df.append(dd)
        counter += 1
    df = pd.DataFrame(df)
    for i in range(1, 6):
        df.at[df[df['num{}'.format(i)] == 10].index, 'num{}'.format(i)] = 0
    for i in range(1, 6):
        df.at[df['num{}'.format(i)].isnull(), 'num{}'.format(i)] = 10
    for i in range(1, 6):
        for j in df['bbox{}'.format(i)][df['bbox{}'.format(i)].isnull()].index:
            df.at[j, 'bbox{}'.format(i+1)] = (0,0,0,0)
    
    df = df.dropna()
    df = df.reset_index(drop=True)

    return df
    
    #df.to_csv('{}.csv'.format(dataset))

In [ ]:
data_ = create_dateset('train', meta_data)

 30%|██▉       | 9990/33402 [01:03<02:19, 167.24it/s]

In [ ]:
data_

,filename,len,num1,bbox1,num2,bbox2,num3,bbox3,num4,bbox4,num5,bbox5
0,1.png,2,1,"(246, 77, 81, 219)",9,"(323, 81, 96, 219)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
1,2.png,2,2,"(77, 29, 23, 32)",3,"(98, 25, 26, 32)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
2,3.png,2,2,"(17, 5, 8, 15)",5,"(25, 5, 9, 15)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
3,4.png,2,9,"(57, 13, 15, 34)",3,"(72, 13, 13, 34)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
4,5.png,2,3,"(52, 7, 21, 46)",1,"(74, 10, 15, 46)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996.png,2,7,"(41, 14, 16, 29)",2,"(57, 13, 17, 29)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
9996,9997.png,2,6,"(46, 11, 22, 42)",7,"(71, 9, 25, 42)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
9997,9998.png,2,1,"(28, 5, 11, 29)",3,"(35, 5, 17, 29)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"
9998,9999.png,2,7,"(32, 10, 5, 20)",5,"(38, 12, 7, 20)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)",-1,"(0, 0, 0, 0)"


In [ ]:
def convert_df (df):

  newdf = {}
  filename = []
  bboxes = []
  labeldict = {}

  num_list = ["num1", "num2", "num3", "num4", "num5"]

  for indx, row in df.iterrows():

    labels = []

    for col in num_list:

      if row[col] != -1:

        filename.append(row["filename"])
        labels.append(row[col])
        bboxes.append(list(row["bbox"+ col[-1]]))

    labeldict[row["filename"]] = labels

  newdf["image_id"] = filename
  newdf["bboxes"] =  bboxes

  df_ = pd.DataFrame(newdf)

  return df_, labeldict

In [ ]:
data2, digits = convert_df(data_)

In [ ]:
digits["2.png"]

[2, 3]

In [ ]:
data2

,image_id,bboxes
0,1.png,"[246, 77, 81, 219]"
1,1.png,"[323, 81, 96, 219]"
2,2.png,"[77, 29, 23, 32]"
3,2.png,"[98, 25, 26, 32]"
4,3.png,"[17, 5, 8, 15]"
...,...,...
11020,4999.png,"[19, 2, 9, 24]"
11021,4999.png,"[27, 2, 7, 24]"
11022,5000.png,"[147, 34, 34, 58]"
11023,5000.png,"[179, 27, 33, 58]"


In [ ]:
data3 = data2.groupby("image_id")["bboxes"].apply(list).reset_index(name="bboxes")

In [ ]:
data3

,image_id,bboxes
0,1.png,"[[246, 77, 81, 219], [323, 81, 96, 219]]"
1,10.png,"[[25, 6, 9, 27], [34, 4, 14, 27]]"
2,100.png,"[[18, 1, 5, 22], [23, 0, 9, 22], [32, 0, 10, 22]]"
3,1000.png,"[[17, 1, 4, 18], [21, 1, 6, 18]]"
4,1001.png,"[[36.0, 4.0, 14.0, 31.0]]"
...,...,...
4995,995.png,"[[52, 20, 9, 22], [60, 18, 9, 22]]"
4996,996.png,"[[34, 10, 15, 23], [46, 9, 13, 23]]"
4997,997.png,"[[21, 4, 14, 23], [35, 4, 14, 23]]"
4998,998.png,"[[15, 3, 8, 12], [26, 5, 5, 12]]"


In [ ]:
df_train, df_validation = model_selection.train_test_split(data3, test_size=0.2, random_state = 42, shuffle = True)

df_train = df_train.reset_index(drop = True)
df_validation = df_validation.reset_index(drop = True)

In [ ]:
DATA_PATH = "/content/gdrive/MyDrive/Sample/resized_300"
OUTPUT_PATH = "/content/gdrive/MyDrive/House_Numbers_300"

In [ ]:
def process_data (data, m_data, labels, data_t="train"):

  counter = 1

  for _, row in tqdm(data.iterrows(), total=len(data)):

    try:

      image_name = row["image_id"]
      bounding_boxes = row["bboxes"]
      yolo_data = []

      counter = 0

      for bbox in bounding_boxes:
        x = bbox[0]
        y = bbox[1]
        w = bbox[2]
        h = bbox[3]

        x_center = x + w/2
        y_center = y + h/2

        x_center /= m_data["image_width"][m_data["image_names"].index(image_name)]
        y_center /= m_data["image_height"][m_data["image_names"].index(image_name)]
        w /= m_data["image_width"][m_data["image_names"].index(image_name)]
        h /= m_data["image_height"][m_data["image_names"].index(image_name)]

        yolo_data.append([labels[image_name][counter], x_center, y_center, w, h])

        counter += 1

      yolo_data = np.array(yolo_data)
      np.savetxt(
          os.path.join(OUTPUT_PATH, f"labels/{data_t}/{image_name[:-4]}.txt"),
                      yolo_data,
                      fmt=["%d", "%f", "%f", "%f", "%f"]
                    )
      shutil.copyfile(
          os.path.join(DATA_PATH, f"{image_name}"),
          os.path.join(OUTPUT_PATH, f"images/{data_t}/{image_name}")
      )

    except:

      counter += 1

  print(counter)
      

In [ ]:
process_data(df_validation, meta_data, digits, data_t = "validation")



  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 3/2000 [00:00<03:28,  9.56it/s]

  0%|          | 7/2000 [00:00<03:21,  9.89it/s]

  0%|          | 10/2000 [00:01<03:44,  8.87it/s]

  1%|          | 17/2000 [00:01<03:05, 10.69it/s]

  1%|          | 22/2000 [00:01<02:23, 13.78it/s]

  1%|▏         | 29/2000 [00:01<01:48, 18.14it/s]

  2%|▏         | 33/2000 [00:02<02:13, 14.79it/s]

  2%|▏         | 38/2000 [00:02<02:13, 14.68it/s]

  2%|▏         | 42/2000 [00:02<02:18, 14.11it/s]

  2%|▏         | 45/2000 [00:03<02:47, 11.69it/s]

  2%|▏         | 47/2000 [00:03<03:50,  8.48it/s]

  2%|▏         | 49/2000 [00:03<04:11,  7.74it/s]

  3%|▎         | 55/2000 [00:03<03:06, 10.44it/s]

  3%|▎         | 61/2000 [00:04<02:35, 12.48it/s]

  3%|▎         | 64/2000 [00:04<03:01, 10.67it/s]

  3%|▎         | 66/2000 [00:04<03:57,  8.16it/s]

  4%|▎         | 72/2000 [00:05<03:17,  9.74it/s]

  4%|▍         | 77/2000 [00:05<03:07, 10.24it/s]

  4%|▍         | 82/2000 [00:05<02:47, 1

3


In [ ]:
process_data(df_train, meta_data, digits, data_t = "train")



  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 2/8000 [00:00<21:42,  6.14it/s]

  0%|          | 3/8000 [00:00<26:21,  5.06it/s]

  0%|          | 7/8000 [00:00<19:31,  6.82it/s]

  0%|          | 12/8000 [00:01<16:37,  8.01it/s]

  0%|          | 18/8000 [00:01<12:18, 10.80it/s]

  0%|          | 26/8000 [00:01<09:07, 14.57it/s]

  0%|          | 31/8000 [00:01<11:09, 11.90it/s]

  0%|          | 35/8000 [00:02<11:08, 11.91it/s]

  0%|          | 40/8000 [00:02<08:48, 15.07it/s]

  1%|          | 43/8000 [00:02<10:35, 12.53it/s]

  1%|          | 50/8000 [00:02<08:02, 16.49it/s]

  1%|          | 55/8000 [00:02<06:28, 20.47it/s]

  1%|          | 59/8000 [00:03<07:52, 16.80it/s]

  1%|          | 63/8000 [00:03<09:28, 13.96it/s]

  1%|          | 68/8000 [00:04<09:33, 13.83it/s]

  1%|          | 71/8000 [00:04<15:33,  8.49it/s]

  1%|          | 77/8000 [00:04<11:36, 11.38it/s]

  1%|          | 80/8000 [00:05<12:17, 10.73it/s]

  1%|          | 83/8000 [00:05<12:49, 10

3


In [ ]:
!ls /content/gdrive/MyDrive/Sample/resized_300/

10000.png  12321.png  14642.png  16963.png  19283.png  21602.png  23923.png
10001.png  12322.png  14643.png  16964.png  19284.png  21603.png  23924.png
10002.png  12323.png  14644.png  16965.png  19285.png  21604.png  23925.png
10003.png  12324.png  14645.png  16966.png  19286.png  21605.png  23926.png
10004.png  12325.png  14646.png  16967.png  19287.png  21606.png  23927.png
10005.png  12326.png  14647.png  16968.png  19288.png  21607.png  23928.png
10006.png  12327.png  14648.png  16969.png  19289.png  21608.png  23929.png
10007.png  12328.png  14649.png  1696.png   1928.png   21609.png  2392.png
10008.png  12329.png  1464.png	 16970.png  19290.png  2160.png   23930.png
10009.png  1232.png   14650.png  16971.png  19291.png  21610.png  23931.png
1000.png   12330.png  14651.png  16972.png  19292.png  21611.png  23932.png
10010.png  12331.png  14652.png  16973.png  19293.png  21612.png  23933.png
10011.png  12332.png  14653.png  16974.png  19294.png  21613.png  23934.png
10012.png  123

In [ ]:
!unzip /content/gdrive/MyDrive/Sample/resized_300.zip -d /content/gdrive/MyDrive/Sample/

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5499.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/55.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/550.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5500.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5501.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5502.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5503.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5504.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5505.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5506.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5507.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5508.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/5509.png  
  inflating: /content/gdrive/MyDrive/Sample/resized_300/551.png  
  inflating: /c